# Downloading ISS-LIS files for particular user input time intervals using an API calls

**Earthdata API is used to access and query NASA data, for details: https://pypi.org/project/earthdata/**

In [1]:
import earthdata
from earthdata import Auth, Store, DataCollections, DataGranules

**Note here we don't need NASA authentication as we are accessing the files directly from GHRC server*

**We can now search for collections using a pythonic API client for CMR**

In [2]:
###Locate DAAC (in oue case it is GHRC-DAAC to access LIS files)###
#Query = DataCollections().daac("GHRCDAAC")

###Find collections in the mentioned DAAC###
#print(f'Collections found: {Query.hits()}')
#collections = Query.fields(['ShortName']).get(10)

###Printing collection 6 of our interest which has ISS science data###
#collections[6]

### Input start time of interest

In [3]:
start_time = input('Enter start time in format: YYYY-MM-DD \n')

Enter start time in format: YYYY-MM-DD 
2021-05-02


### Input end time of interest

In [4]:
end_time = input('Enter end time in format: YYYY-MM-DD \n')

Enter end time in format: YYYY-MM-DD 
2021-05-03


**To find LIS granules for given dates and access their metadata using get() method**

In [12]:
### We build our query, note as ISS_LIS data comes 1file per orbit so we can temporally query the data### 
###spatial query not possible###
###The short name for collection was found from cell[2]###

from pprint import pprint
Query = DataGranules().short_name('isslis_v2_nqc').temporal(start_time,end_time)

###We get all metadata records###
granules = Query.get(2)

granules

[Collection: {'ShortName': 'isslis_v2_nqc', 'Version': '2'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -75.131, 'EastBoundingCoordinate': -101.506, 'NorthBoundingCoordinate': 51.84, 'SouthBoundingCoordinate': -51.706}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-05-01T22:33:54Z', 'EndingDateTime': '2021-05-02T00:06:47Z'}}
 Size(MB): 2.02
 Data: ['https://data.ghrc.earthdata.nasa.gov/ghrcw-protected/isslis_v2_nqc__2/202105/ISS_LIS_SC_V2.1_20210501_223343_NQC.hdf', 's3://ghrcw-protected/isslis_v2_nqc__2/202105/ISS_LIS_SC_V2.1_20210501_223343_NQC.hdf'],
 Collection: {'ShortName': 'isslis_v2_nqc', 'Version': '2'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -75.131, 'EastBoundingCoordinate': -101.506, 'NorthBoundingCoordinate': 51.84, 'SouthBoundingCoordinate': -51.706}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': 

In [7]:
###explore granules metadata###
#[display(granule) for granule in granules]

**Now lets try to extract data URLS from the metadata of each datasets of our interest**

In [9]:
data_links = [granule.data_links() for granule in granules]

#print(data_links)
#type(data_links)

**OOPS!! we are not able to access the GHRC DAAC server to directly download the files using get() method from earthdata library so lets take a long cut (atleast for time being)**

**'https' url for GHRC-DAAC server and 's3'for AWS are sorted...**

In [10]:
##covert list to dataframe .. a long cut again :)
import pandas as pd
df = pd.DataFrame(data_links, columns = ['https' , 'sf3'])

##converted dataframe back to list sorting hhtps urls
inprem = df['https'].values.tolist()
#print(inprem)
 
##Sort nc or h5 from list : work in progress
#sort_order = ['nc']
#inprem.sort(key = lambda i: sort_order) # works in python 3
 
##printing result
#print ("The sorted list is : " + str(inprem))

***Finally now, https urls for GHRC-DAAC server are opened using loop and temporarily sorted files are downloaded ;)***

In [ ]:
##Works https url opens and data is downloaded to downloads as default location

import webbrowser
#url_list = ['https://data.ghrc.earthdata.nasa.gov/ghrcw-protected/isslis_v2_nqc__2/202002/ISS_LIS_SC_V2.1_20200201_225844_NQC.hdf', 'https://data.ghrc.earthdata.nasa.gov/ghrcw-protected/isslis_v2_nqc__2/202002/ISS_LIS_SC_V2.1_20200201_225844_NQC.nc']

for url in inprem:
    response = webbrowser.open(url)

***THE CODE ENDs***

.

**Issues yet to be addressed:**
1) Not able to access the files from locall DAAC server directly using get() method
(By default the AWS links gets accessed and no file is downloaded)

2) Have used URL open method to download files but all files are saved in downloads: How to change the directory ?

NOT WORKING: Download URL method not working

#import requests
#response = requests.get('https://data.ghrc.earthdata.nasa.gov/ghrcw-protected/isslis_v2_nqc__2/202002/ISS_LIS_SC_V2.1_20200201_225844_NQC.hdf')